In [1]:
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define file paths
trj = "project3/asp_trx/merged_trajectory.xtc"
top = "project3/asp_trx/asp900ns.tpr"

# Initialize MDAnalysis Universe
u = mda.Universe(top, trj)

# Select the micelle atoms
micelle_atoms = u.select_atoms("resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)")

# Function to calculate the moments of inertia and ellipticity
def calculate_moments_and_ellipticity(atom_group):
    positions = atom_group.positions - atom_group.center_of_mass()  # Center positions to COM
    masses = atom_group.masses[:, np.newaxis]  # Masses as column vector
    inertia_tensor = np.zeros((3, 3))  # Initialize 3x3 inertia tensor

    # Calculate inertia tensor
    for i, coord in enumerate(positions):
        mass = masses[i, 0]
        inertia_tensor[0, 0] += mass * (coord[1]**2 + coord[2]**2)
        inertia_tensor[1, 1] += mass * (coord[0]**2 + coord[2]**2)
        inertia_tensor[2, 2] += mass * (coord[0]**2 + coord[1]**2)
        inertia_tensor[0, 1] -= mass * coord[0] * coord[1]
        inertia_tensor[0, 2] -= mass * coord[0] * coord[2]
        inertia_tensor[1, 2] -= mass * coord[1] * coord[2]

    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]

    # Calculate eigenvalues (principal moments of inertia)
    principal_moments = np.linalg.eigvalsh(inertia_tensor)
    principal_moments = np.sort(principal_moments)  # Sort ascending (I_min, ..., I_max)
    
    # Calculate ellipticity
    ellipticity = principal_moments[-1] / principal_moments[0]  # I_max / I_min
    return principal_moments, ellipticity

# Classify shape based on principal moments of inertia
def classify_shape(principal_moments):
    I_x, I_y, I_z = principal_moments
    if np.isclose(I_x, I_y) and np.isclose(I_y, I_z):
        return "Spherical"
    elif np.isclose(I_x, I_y) and I_x > I_z:
        return "Prolate"
    elif np.isclose(I_y, I_z) and I_y > I_x:
        return "Oblate"
    else:
        return "Triaxial"

# Variables to store results
times = []
ellipticities = []
shapes = []
principal_moments_list = []

# Loop through trajectory and calculate inertia properties
print("Processing trajectory to calculate moments of inertia, ellipticity, and classify shapes...")
for ts in tqdm(u.trajectory[::10], desc="Frames Processed", unit="frame", ncols=100):  # Skip every 10 ns
    time_ns = ts.time / 1000  # Convert time to nanoseconds

    # Process only frames between 700 ns and 1000 ns
    if 700 <= time_ns <= 1000:
        times.append(time_ns)

        # Calculate moments of inertia and ellipticity
        principal_moments, ellipticity = calculate_moments_and_ellipticity(micelle_atoms)
        ellipticities.append(ellipticity)
        principal_moments_list.append(principal_moments)

        # Classify shape
        shape = classify_shape(principal_moments)
        shapes.append(shape)

# Calculate average and standard deviation of ellipticity
average_ellipticity = np.mean(ellipticities)
std_ellipticity = np.std(ellipticities)

# Output results
print("\n--- Results from 700 ns to 1000 ns ---")
print(f"Average Ellipticity: {average_ellipticity:.2f}")
print(f"Standard Deviation of Ellipticity: {std_ellipticity:.2f}")

# Shape classification distribution
shape_distribution = {shape: shapes.count(shape) for shape in set(shapes)}
print("Shape Distribution:")
for shape, count in shape_distribution.items():
    print(f"  {shape}: {count} occurrences")


Processing trajectory to calculate moments of inertia, ellipticity, and classify shapes...


Frames Processed: 100%|███████████████████████████████████| 10201/10201 [01:28<00:00, 115.64frame/s]


--- Results from 700 ns to 1000 ns ---
Average Ellipticity: 1.93
Standard Deviation of Ellipticity: 0.32
Shape Distribution:
  Triaxial: 3001 occurrences


In [2]:
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm

# Define file paths
trj = "project3/asp_trx/merged_trajectory.xtc"
top = "project3/asp_trx/asp900ns.tpr"

# Initialize MDAnalysis Universe
u = mda.Universe(top, trj)

# Select the micelle atoms
micelle_atoms = u.select_atoms("resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)")

# Function to calculate the moments of inertia
def calculate_moments_of_inertia(atom_group):
    positions = atom_group.positions - atom_group.center_of_mass()  # Center positions to COM
    masses = atom_group.masses[:, np.newaxis]  # Masses as column vector
    inertia_tensor = np.zeros((3, 3))  # Initialize 3x3 inertia tensor

    # Calculate inertia tensor
    for i, coord in enumerate(positions):
        mass = masses[i, 0]
        inertia_tensor[0, 0] += mass * (coord[1]**2 + coord[2]**2)
        inertia_tensor[1, 1] += mass * (coord[0]**2 + coord[2]**2)
        inertia_tensor[2, 2] += mass * (coord[0]**2 + coord[1]**2)
        inertia_tensor[0, 1] -= mass * coord[0] * coord[1]
        inertia_tensor[0, 2] -= mass * coord[0] * coord[2]
        inertia_tensor[1, 2] -= mass * coord[1] * coord[2]

    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]

    # Calculate eigenvalues (principal moments of inertia)
    principal_moments = np.linalg.eigvalsh(inertia_tensor)
    principal_moments = np.sort(principal_moments)  # Sort ascending (I_min, ..., I_max)
    return principal_moments

# Classify shape based on principal moments of inertia
def classify_shape(principal_moments):
    I_x, I_y, I_z = principal_moments
    if np.isclose(I_x, I_y) and np.isclose(I_y, I_z):
        return "Spherical"
    elif np.isclose(I_x, I_y) and I_x > I_z:
        return "Prolate"
    elif np.isclose(I_y, I_z) and I_y > I_x:
        return "Oblate"
    else:
        return "Triaxial"

# Variables to store results
shapes = []

# Loop through trajectory and calculate shape
print("Processing trajectory to calculate moments of inertia and determine shape...")
for ts in tqdm(u.trajectory, desc="Frames Processed", unit="frame", ncols=100):
    # Calculate moments of inertia
    principal_moments = calculate_moments_of_inertia(micelle_atoms)

    # Classify shape
    shape = classify_shape(principal_moments)
    shapes.append(shape)

# Output summary results
shape_distribution = {shape: shapes.count(shape) for shape in set(shapes)}
print("\n--- Shape Distribution Across Entire Trajectory ---")
for shape, count in shape_distribution.items():
    print(f"  {shape}: {count} occurrences")


Processing trajectory to calculate moments of inertia and determine shape...


Frames Processed: 100%|██████████████████████████████████| 102001/102001 [30:12<00:00, 56.29frame/s]


--- Shape Distribution Across Entire Trajectory ---
  Triaxial: 102001 occurrences


In [3]:
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define file paths
trj = 'project3/aten_trx/merged_aten.xtc'
top = 'project3/aten_trx/aten900ns.tpr'

# Initialize MDAnalysis Universe
u = mda.Universe(top, trj)

# Select the micelle atoms
micelle_atoms = u.select_atoms("resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)")

# Function to calculate the moments of inertia and ellipticity
def calculate_moments_and_ellipticity(atom_group):
    positions = atom_group.positions - atom_group.center_of_mass()  # Center positions to COM
    masses = atom_group.masses[:, np.newaxis]  # Masses as column vector
    inertia_tensor = np.zeros((3, 3))  # Initialize 3x3 inertia tensor

    # Calculate inertia tensor
    for i, coord in enumerate(positions):
        mass = masses[i, 0]
        inertia_tensor[0, 0] += mass * (coord[1]**2 + coord[2]**2)
        inertia_tensor[1, 1] += mass * (coord[0]**2 + coord[2]**2)
        inertia_tensor[2, 2] += mass * (coord[0]**2 + coord[1]**2)
        inertia_tensor[0, 1] -= mass * coord[0] * coord[1]
        inertia_tensor[0, 2] -= mass * coord[0] * coord[2]
        inertia_tensor[1, 2] -= mass * coord[1] * coord[2]

    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]

    # Calculate eigenvalues (principal moments of inertia)
    principal_moments = np.linalg.eigvalsh(inertia_tensor)
    principal_moments = np.sort(principal_moments)  # Sort ascending (I_min, ..., I_max)
    
    # Calculate ellipticity
    ellipticity = principal_moments[-1] / principal_moments[0]  # I_max / I_min
    return principal_moments, ellipticity

# Classify shape based on principal moments of inertia
def classify_shape(principal_moments):
    I_x, I_y, I_z = principal_moments
    if np.isclose(I_x, I_y) and np.isclose(I_y, I_z):
        return "Spherical"
    elif np.isclose(I_x, I_y) and I_x > I_z:
        return "Prolate"
    elif np.isclose(I_y, I_z) and I_y > I_x:
        return "Oblate"
    else:
        return "Triaxial"

# Variables to store results
times = []
ellipticities = []
shapes = []
principal_moments_list = []

# Loop through trajectory and calculate inertia properties
print("Processing trajectory to calculate moments of inertia, ellipticity, and classify shapes...")
for ts in tqdm(u.trajectory[::10], desc="Frames Processed", unit="frame", ncols=100):  # Skip every 10 ns
    time_ns = ts.time / 1000  # Convert time to nanoseconds

    # Process only frames between 700 ns and 1000 ns
    if 700 <= time_ns <= 1000:
        times.append(time_ns)

        # Calculate moments of inertia and ellipticity
        principal_moments, ellipticity = calculate_moments_and_ellipticity(micelle_atoms)
        ellipticities.append(ellipticity)
        principal_moments_list.append(principal_moments)

        # Classify shape
        shape = classify_shape(principal_moments)
        shapes.append(shape)

# Calculate average and standard deviation of ellipticity
average_ellipticity = np.mean(ellipticities)
std_ellipticity = np.std(ellipticities)

# Output results
print("\n--- Results from 700 ns to 1000 ns ---")
print(f"Average Ellipticity: {average_ellipticity:.2f}")
print(f"Standard Deviation of Ellipticity: {std_ellipticity:.2f}")

# Shape classification distribution
shape_distribution = {shape: shapes.count(shape) for shape in set(shapes)}
print("Shape Distribution:")
for shape, count in shape_distribution.items():
    print(f"  {shape}: {count} occurrences")


Processing trajectory to calculate moments of inertia, ellipticity, and classify shapes...


Frames Processed: 100%|████████████████████████████████████| 10201/10201 [01:50<00:00, 92.14frame/s]


--- Results from 700 ns to 1000 ns ---
Average Ellipticity: 3.78
Standard Deviation of Ellipticity: 0.22
Shape Distribution:
  Triaxial: 3001 occurrences


In [ ]:
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm

# Define file paths
trj = 'project3/aten_trx/merged_aten.xtc'
top = 'project3/aten_trx/aten900ns.tpr'

# Initialize MDAnalysis Universe
u = mda.Universe(top, trj)

# Select the micelle atoms
micelle_atoms = u.select_atoms("resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)")

# Function to calculate the moments of inertia
def calculate_moments_of_inertia(atom_group):
    positions = atom_group.positions - atom_group.center_of_mass()  # Center positions to COM
    masses = atom_group.masses[:, np.newaxis]  # Masses as column vector
    inertia_tensor = np.zeros((3, 3))  # Initialize 3x3 inertia tensor

    # Calculate inertia tensor
    for i, coord in enumerate(positions):
        mass = masses[i, 0]
        inertia_tensor[0, 0] += mass * (coord[1]**2 + coord[2]**2)
        inertia_tensor[1, 1] += mass * (coord[0]**2 + coord[2]**2)
        inertia_tensor[2, 2] += mass * (coord[0]**2 + coord[1]**2)
        inertia_tensor[0, 1] -= mass * coord[0] * coord[1]
        inertia_tensor[0, 2] -= mass * coord[0] * coord[2]
        inertia_tensor[1, 2] -= mass * coord[1] * coord[2]

    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]

    # Calculate eigenvalues (principal moments of inertia)
    principal_moments = np.linalg.eigvalsh(inertia_tensor)
    principal_moments = np.sort(principal_moments)  # Sort ascending (I_min, ..., I_max)
    return principal_moments

# Classify shape based on principal moments of inertia
def classify_shape(principal_moments):
    I_x, I_y, I_z = principal_moments
    if np.isclose(I_x, I_y) and np.isclose(I_y, I_z):
        return "Spherical"
    elif np.isclose(I_x, I_y) and I_x > I_z:
        return "Prolate"
    elif np.isclose(I_y, I_z) and I_y > I_x:
        return "Oblate"
    else:
        return "Triaxial"

# Variables to store results
shapes = []

# Loop through trajectory and calculate shape
print("Processing trajectory to calculate moments of inertia and determine shape...")
for ts in tqdm(u.trajectory, desc="Frames Processed", unit="frame", ncols=100):
    # Calculate moments of inertia
    principal_moments = calculate_moments_of_inertia(micelle_atoms)

    # Classify shape
    shape = classify_shape(principal_moments)
    shapes.append(shape)

# Output summary results
shape_distribution = {shape: shapes.count(shape) for shape in set(shapes)}
print("\n--- Shape Distribution Across Entire Trajectory ---")
for shape, count in shape_distribution.items():
    print(f"  {shape}: {count} occurrences")

In [4]:
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define file paths
trj = 'project3/fel_trx/merged_fel.xtc'
top = 'project3/fel_trx/fel900ns.tpr'

# Initialize MDAnalysis Universe
u = mda.Universe(top, trj)

# Select the micelle atoms
micelle_atoms = u.select_atoms("resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)")

# Function to calculate the moments of inertia and ellipticity
def calculate_moments_and_ellipticity(atom_group):
    positions = atom_group.positions - atom_group.center_of_mass()  # Center positions to COM
    masses = atom_group.masses[:, np.newaxis]  # Masses as column vector
    inertia_tensor = np.zeros((3, 3))  # Initialize 3x3 inertia tensor

    # Calculate inertia tensor
    for i, coord in enumerate(positions):
        mass = masses[i, 0]
        inertia_tensor[0, 0] += mass * (coord[1]**2 + coord[2]**2)
        inertia_tensor[1, 1] += mass * (coord[0]**2 + coord[2]**2)
        inertia_tensor[2, 2] += mass * (coord[0]**2 + coord[1]**2)
        inertia_tensor[0, 1] -= mass * coord[0] * coord[1]
        inertia_tensor[0, 2] -= mass * coord[0] * coord[2]
        inertia_tensor[1, 2] -= mass * coord[1] * coord[2]

    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]

    # Calculate eigenvalues (principal moments of inertia)
    principal_moments = np.linalg.eigvalsh(inertia_tensor)
    principal_moments = np.sort(principal_moments)  # Sort ascending (I_min, ..., I_max)
    
    # Calculate ellipticity
    ellipticity = principal_moments[-1] / principal_moments[0]  # I_max / I_min
    return principal_moments, ellipticity

# Classify shape based on principal moments of inertia
def classify_shape(principal_moments):
    I_x, I_y, I_z = principal_moments
    if np.isclose(I_x, I_y) and np.isclose(I_y, I_z):
        return "Spherical"
    elif np.isclose(I_x, I_y) and I_x > I_z:
        return "Prolate"
    elif np.isclose(I_y, I_z) and I_y > I_x:
        return "Oblate"
    else:
        return "Triaxial"

# Variables to store results
times = []
ellipticities = []
shapes = []
principal_moments_list = []

# Loop through trajectory and calculate inertia properties
print("Processing trajectory to calculate moments of inertia, ellipticity, and classify shapes...")
for ts in tqdm(u.trajectory[::10], desc="Frames Processed", unit="frame", ncols=100):  # Skip every 10 ns
    time_ns = ts.time / 1000  # Convert time to nanoseconds

    # Process only frames between 700 ns and 1000 ns
    if 700 <= time_ns <= 1000:
        times.append(time_ns)

        # Calculate moments of inertia and ellipticity
        principal_moments, ellipticity = calculate_moments_and_ellipticity(micelle_atoms)
        ellipticities.append(ellipticity)
        principal_moments_list.append(principal_moments)

        # Classify shape
        shape = classify_shape(principal_moments)
        shapes.append(shape)

# Calculate average and standard deviation of ellipticity
average_ellipticity = np.mean(ellipticities)
std_ellipticity = np.std(ellipticities)

# Output results
print("\n--- Results from 700 ns to 1000 ns ---")
print(f"Average Ellipticity: {average_ellipticity:.2f}")
print(f"Standard Deviation of Ellipticity: {std_ellipticity:.2f}")

# Shape classification distribution
shape_distribution = {shape: shapes.count(shape) for shape in set(shapes)}
print("Shape Distribution:")
for shape, count in shape_distribution.items():
    print(f"  {shape}: {count} occurrences")


Processing trajectory to calculate moments of inertia, ellipticity, and classify shapes...


Frames Processed: 100%|█████████████████████████████████████| 9270/9270 [01:31<00:00, 101.51frame/s]


--- Results from 700 ns to 1000 ns ---
Average Ellipticity: 1.59
Standard Deviation of Ellipticity: 0.06
Shape Distribution:
  Triaxial: 2070 occurrences
